In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


def image_analysis_x(
    root,
    filename="MOT_loading_121025_take4.avi",
    out_prefix="loading_JY_2_PHYS348_tk4",
):
    """
    Python port of image_analysis_x.m

    Parameters
    ----------
    root : str or Path
        Directory containing the AVI file.
    filename : str
        Name of the video file.
    out_prefix : str
        Prefix for the output CSV file.
    """
    root = Path(root)
    video_path = root / filename

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video file: {video_path}")

    # Use actual FPS if available; otherwise fall back to your hard-coded value
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 34.815
    print(f"FPS: {fps:.3f}")

    # Background frame: MATLAB used frame number 70 (1-based)
    F_bg = 70
    bg_frame_index = F_bg - 1  # convert to 0-based

    # Helper: grab a frame as grayscale float
    def get_frame(frame_index: int) -> np.ndarray:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ok, frame = cap.read()
        if not ok:
            raise ValueError(f"Could not read frame {frame_index}")
        # Convert BGR -> grayscale, then to float64 for safe subtraction
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return gray.astype(np.float64)

    # Background image
    bg = get_frame(bg_frame_index)

    # Physics / MOT parameters (copied from MATLAB, though not all used explicitly)
    lambda_rb = 780.241368271e-9  # [m]
    Gamma = 2 * np.pi * 6.066e6   # [Hz]
    tau = 26.2348e-9              # [s]

    IIsat = 0.490 / 1.671         # I / Isat, dimensionless

    # Geometry (still in inches, as in your MATLAB; ratio is dimensionless)
    radius = 1.0                  # lens radius [in]
    dist = 19.625                 # distance MOT–lens [in]

    te = 20.392e-3                # exposure time [s]
    qbar = 2.967                  # counts per photon-ish factor

    det_rat = 3.0                 # Δ / Γ
    rho_ee = 0.5 * IIsat / (4 * det_rat**2 + IIsat + 1)  # excited-state fraction

    omega = np.pi * (radius / dist) ** 2
    eta = omega / (4 * np.pi)     # collection efficiency

    # ROI: convert your MATLAB indices (500–700, 600–800) to 0-based
    # MATLAB: rows y = 600:800, cols x = 500:700
    # Python: indices 599:800 and 499:700 to hit essentially the same region
    x1, x2 = 499, 700   # inclusive in MATLAB; Python slice will use x2 as upper bound
    y1, y2 = 599, 800

    times = []
    numbers = []

    # Loop over frames: MATLAB uses for i = 1:290, F = i*10
    for i in range(1, 291):
        F = i * 10
        frame_index = F - 1  # MATLAB -> Python

        try:
            pic = get_frame(frame_index)
        except ValueError:
            print(f"Stopping early: could not read frame {frame_index}")
            break

        # Background-subtracted MOT signal
        good = pic - bg

        # Extract region of interest
        roi = good[y1:y2, x1:x2]  # y first, then x

        # Total counts in ROI
        Nc = roi.sum()

        # Atom number (identical formula to MATLAB)
        # Na = Nc * qbar / (eta * te / tau * rho_ee)
        Na = Nc * qbar / ((eta * te / tau) * rho_ee)

        # Time corresponding to this frame
        Ta = F / fps

        times.append(Ta)
        numbers.append(Na)

    cap.release()

    times = np.array(times)
    numbers = np.array(numbers)

    # Save to CSV (can be opened by Excel)
    out_csv = root / f"{out_prefix}.csv"
    df = pd.DataFrame({"time_s": times, "N_atoms": numbers})
    df.to_csv(out_csv, index=False)
    print(f"Saved MOT loading data to {out_csv}")

    # Plot N(t)
    plt.figure()
    plt.plot(times, numbers)
    plt.xlabel("Time [s]")
    plt.ylabel("Number of atoms")
    plt.title("MOT Explosion - Intensity vs. Time")
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    # Example usage: adjust root to wherever your AVI lives now
    image_analysis_x(r"C:\Users\johnny\Desktop\PHYS348")
